## Install Llama3

In [123]:
#installed ollama to my device, then using the command line ran llama3, 
!pip install requests
import requests

## Testing

In [126]:
#Defining prompt
model = 'llama3'
prompt= "Tell me about Large Language Models"
url = 'http://localhost:11434/api/chat'
data = {"model": model,"messages": [{"role": "user", "content": prompt}],"stream": False}
response= requests.post(url,json=data)

#print response
print(response.json()['message']['content'])

A fascinating topic!

Large Language Models (LLMs) are a type of artificial intelligence (AI) model that have revolutionized the field of natural language processing (NLP). They are trained on massive amounts of text data and can process and generate human-like language, making them incredibly useful for various applications.

**What is a Large Language Model?**

A LLM is a type of neural network-based AI model designed to process and generate large volumes of text data. These models are typically trained on massive datasets of text, such as books, articles, and web pages, using techniques like masked language modeling, next sentence prediction, and supervised learning.

**Key Characteristics:**

1. **Scale**: LLMs are characterized by their enormous size, measured in terms of the number of parameters, which can range from hundreds of millions to billions.
2. **Depth**: They have multiple layers of neural network architecture, allowing them to capture complex patterns and relationships

## Import modules

In [150]:
#importing modules
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

In [131]:
# naming file as a variable
file = "genreLyrics.csv"

In [133]:
#reading and checking df
df = pd.read_csv(file, engine = 'python', sep = "\t", encoding = 'utf8')
df.head()

,Unnamed: 0,genre,lyrics
0,76301,Rock,"Hey, if you were right I'd chase away\r\nAll t..."
1,293332,Rock,There's something about the way we fit\r\nTher...
2,70683,Electronic,One drop in the ocean\r\nCould be that magic p...
3,209590,Rock,I'm so tired of being here\r\nSuppressed by al...
4,116010,Hip-Hop,"Yeah, what, Vast Aire,; Shell Shock..\r\nIt's ..."


## 1. Zero-Shot Prompt Function

In [200]:
#classification of lyrics using the zero shot prompt method
def zero_shot_prompt(lyric, model = 'llama3'):
    
    prompt= f'''Classify these song lyrics with their associated genre. Only respond with a one-word genre label. Like rock, hip-hop, jazz etc. 

Lyrics: 
\"\"\"
{lyric}
\"\"\"
Genre:'''
    
    url = 'http://localhost:11434/api/chat'
    data = {"model": model,"messages": [{"role": "user", "content": prompt}],"stream": False}
    response= requests.post(url,json=data)
    return response.json()['message']['content'].strip()

In [202]:
df['zero_predicted_genre'] = df['lyrics'].head(5).apply(zero_shot_prompt)
print(df['zero_predicted_genre'].head(5))

0      Indie
1       Rock
2        Pop
3        Emo
4    Hip-Hop
Name: zero_predicted_genre, dtype: object


## 2. Few-Shot Prompt Function

In [141]:
#classification of lyrics using the few shot prompt method
def few_shot_prompt(lyric, model = 'llama3'):
    
    prompt= f'''Classify these song lyrics with their associated genre. Only respond with a one-word genre label, like rock, hip-hop, jazz etc. 

Examples:

Lyrics: 
\"\"\"
Stand up beside the fireplace
Take that look from off your face
\"\"\"
Genre: Rock

Lyrics: 
\"\"\"
It was all laid out for me
But I feel like I'm in love with the pain, fam
\"\"\"
Genre: Grime

Lyrics: 
\"\"\"
Oh you're so naive yet so
How could this be done
\"\"\"
Genre: Indie

Now classify these lyrics:

Lyrics:
\"\"\"
{lyric}
\"\"\"
Genre:'''
    
    url = 'http://localhost:11434/api/chat'
    data = {"model": model,"messages": [{"role": "user", "content": prompt}],"stream": False}
    response= requests.post(url,json=data)
    return response.json()['message']['content'].strip()

In [180]:
df['few_predicted_genre'] = df['lyrics'].head(5).apply(few_shot_prompt)
print(df['few_predicted_genre'].head(5))

0      Indie
1        Rap
2        Pop
3        Emo
4    Hip-Hop
Name: few_predicted_genre, dtype: object


## 3. Calculating the Precision, Recall and F1 Score

In [204]:
#making my smaller set for me to perform these measurements on
test_df = df.head(5).copy()

test_df['genre']
test_df['zero_predicted_genre']
test_df['few_predicted_genre']

y_true = test_df['genre'].str.lower().str.strip()
y_pred1 = test_df['zero_predicted_genre'].str.lower().str.strip()
y_pred2 = test_df['few_predicted_genre'].str.lower().str.strip()

## 4. Zero Shot Precision, Recall and F1 scores

In [207]:
#calculating precision, recall and f1 scores
zero_precision = precision_score(y_true, y_pred1, average = 'weighted', zero_division = 0)
zero_recall = recall_score(y_true, y_pred1, average = 'weighted', zero_division = 0)
zero_f1 = f1_score(y_true, y_pred1, average = 'weighted', zero_division = 0)

print(f'Zero Shot Precision: {zero_precision:.4f}')
print(f'Zero Shot Recall: {zero_recall:.4f}')
print(f'Zero Shot F1: {zero_f1:.4f}')

Zero Shot Precision: 0.8000
Zero Shot Recall: 0.4000
Zero Shot F1: 0.5000


## 5. Few Shot Precision, Recall and F1 scores

In [210]:
#calculating precision, recall and f1 scores
few_precision = precision_score(y_true, y_pred2, average = 'weighted', zero_division = 0)
few_recall = recall_score(y_true, y_pred2, average = 'weighted', zero_division = 0)
few_f1 = f1_score(y_true, y_pred2, average = 'weighted', zero_division = 0)

print(f'Few Shot Precision: {few_precision:.4f}')
print(f'Few Shot Recall: {few_recall:.4f}')
print(f'Few Shot F1: {few_f1:.4f}')

Few Shot Precision: 0.2000
Few Shot Recall: 0.2000
Few Shot F1: 0.2000


In [214]:
#comparing the results between the correct answer, in genre, and the predicted ones
print(test_df[['genre', 'zero_predicted_genre', 'few_predicted_genre']])

        genre zero_predicted_genre few_predicted_genre
0        Rock                Indie               Indie
1        Rock                 Rock                 Rap
2  Electronic                  Pop                 Pop
3        Rock                  Emo                 Emo
4     Hip-Hop              Hip-Hop             Hip-Hop
